Parte 2: Olist

Desafio: Criar um notebook de análise exploratória (como fizemos na nossa aula de hoje) com todas as etapas de coleta, limpeza, análise e visualização com base de dados da Olist.

**Requisitos:**

Formular no mínimo **2 perguntas** (caso queira se desafiar crie pelo menos mais perguntas além das 2 obrigatórias) para responder com suas análises;
Utilizar pelo menos **3 bases de dados da Olist** (caso você deseje criar sua base do zero). Caso deseje continuar utilizando a que criamos em aula, é necessário incluir pelo menos mais **1 tabela** para enriquecer sua análise.

Para responder as perguntas elaboradas usar:

Criar pelo menos **2 gráficos**.
Exporte sua base final em csv.
Submeta uma pasta que contenha:
o arquivo **seu_nome.ipynb** com sua análise exploratória;
a base final criada por você no formato .csv;

In [58]:
# Formular no mínimo **2 perguntas** (caso queira se desafiar crie pelo menos mais perguntas além das 2 obrigatórias) para responder com suas análises;
# Qual o produto mais vendido?
import pandas as pd

df_produtos = pd.read_csv('../para-sala/dados/olist_products_dataset.csv') #mergeado
df_pedidos = pd.read_csv('../para-sala/dados/olist_orders_dataset.csv') #mergeado
df_itens_pedidos = pd.read_csv('../para-sala/dados/olist_order_items_dataset.csv') #mergeado
df_meio_pgto = pd.read_csv('../para-sala/dados/olist_order_payments_dataset.csv')
df_consumidor = pd.read_csv('../para-sala/dados/olist_customers_dataset.csv') #mergeado


In [60]:
columns_drop = ['order_approved_at', 'order_delivered_carrier_date', 'order_estimated_delivery_date']
df_pedidos = df_pedidos.drop(columns_drop, axis=1)
# df_pedidos = df_pedidos.loc[df_pedidos['order_status'] == 'delivered']
# df_pedidos = df_pedidos.reset_index(drop=True)
df_pedidos['order_purchase_timestamp'] = pd.to_datetime(df_pedidos['order_purchase_timestamp'])
df_pedidos['order_delivered_customer_date'] = pd.to_datetime(df_pedidos['order_delivered_customer_date'])
df_pedidos['reference_month'] = df_pedidos['order_purchase_timestamp'].dt.strftime('%Y-%m-01')

In [61]:
df_consumidor = df_consumidor.drop('customer_unique_id', axis=1)

In [62]:
# Qual o produto mais vendido?
df_produtos = df_produtos[['product_id','product_category_name']]
df_produtos = df_produtos.dropna()
df_produtos = df_produtos.reset_index(drop=True)

In [63]:
df_olist_pedidos = pd.merge(df_pedidos, df_itens_pedidos, on=['order_id'], how='inner')
df_olist_produtos_pedidos = pd.merge(df_olist_pedidos, df_produtos, on=['product_id'], how='inner')
df_olist_produtos_pedidos_uf = pd.merge(df_olist_produtos_pedidos, df_consumidor, on=['customer_id'], how='inner')
df_olist = pd.merge(df_meio_pgto, df_olist_produtos_pedidos_uf, on=['order_id'], how='inner')

In [64]:
df_olist.isnull().sum()

order_id                            0
payment_sequential                  0
payment_type                        0
payment_installments                0
payment_value                       0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_delivered_customer_date    2497
reference_month                     0
order_item_id                       0
product_id                          0
seller_id                           0
shipping_limit_date                 0
price                               0
freight_value                       0
product_category_name               0
customer_zip_code_prefix            0
customer_city                       0
customer_state                      0
dtype: int64

In [14]:
# df_olist.to_csv('base_final_s14_olist.csv', index=False)

In [65]:
# df_sazonalidade.sort_values(by='order_id', ascending=False)
df_olist['order_purchase_timestamp'] = pd.to_datetime(df_olist['order_purchase_timestamp'])
df_olist['order_delivered_customer_date'] = pd.to_datetime(df_olist['order_delivered_customer_date'])
df_olist['order_delivered_date'] = df_olist['order_delivered_customer_date'].dt.date
df_olist['shipping_limit_dt'] = pd.to_datetime(df_olist['shipping_limit_date']).dt.date
df_olist.loc[df_olist['order_delivered_date'] > df_olist['shipping_limit_dt'], 'status_entrega'] = 'ATRASADO'
df_olist.loc[df_olist['order_delivered_date'] < df_olist['shipping_limit_dt'], 'status_entrega'] = 'ADIANTADO'
df_olist.loc[df_olist['order_delivered_date'] == df_olist['shipping_limit_dt'], 'status_entrega'] = 'DENTRO DO ESPERADO'

In [66]:
df_olist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115903 entries, 0 to 115902
Data columns (total 23 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   order_id                       115903 non-null  object        
 1   payment_sequential             115903 non-null  int64         
 2   payment_type                   115903 non-null  object        
 3   payment_installments           115903 non-null  int64         
 4   payment_value                  115903 non-null  float64       
 5   customer_id                    115903 non-null  object        
 6   order_status                   115903 non-null  object        
 7   order_purchase_timestamp       115903 non-null  datetime64[ns]
 8   order_delivered_customer_date  113406 non-null  datetime64[ns]
 9   reference_month                115903 non-null  object        
 10  order_item_id                  115903 non-null  int64         
 11  

In [67]:
df_olist = df_olist[['reference_month','order_id','product_id','product_category_name','price','order_status','status_entrega','customer_state','payment_value','payment_type']]

In [68]:
df_olist['status_entrega'] = df_olist['status_entrega'].fillna('NAO IDENTIFICADO')

In [69]:
df_olist['lucro'] = (df_olist['payment_value']) - (df_olist['price'])
df_olist


,reference_month,order_id,product_id,product_category_name,price,order_status,status_entrega,customer_state,payment_value,payment_type,lucro
0,2018-04-01,b81ef226f3fe1789b1e8b2acac839d17,af74cc53dcffc8384b29e7abfa41902b,casa_construcao,79.80,delivered,ATRASADO,MG,99.33,credit_card,19.53
1,2018-06-01,a9810da82917af2d9aefd1278f1dcfa0,a630cc320a8c872f9de830cf121661a3,automotivo,17.00,delivered,ADIANTADO,SP,24.39,credit_card,7.39
2,2017-12-01,25e8ea4e93396b6fa0d3dd708e76c1bd,2028bf1b01cafb2d2b1901fca4083222,perfumaria,56.99,delivered,ADIANTADO,SP,65.71,credit_card,8.72
3,2017-12-01,ba78997921bbcdc1373bb41e913ab953,548e5bfe28edceab6b51fa707cc9556f,cama_mesa_banho,89.90,delivered,ATRASADO,MG,107.78,credit_card,17.88
4,2018-05-01,42fdf880ba16b47b59251dd489d4441a,386486367c1f9d4f587a8864ccb6902b,cama_mesa_banho,113.57,delivered,ATRASADO,SP,128.45,credit_card,14.88
...,...,...,...,...,...,...,...,...,...,...,...
115898,2018-03-01,0406037ad97740d563a178ecc7a2075c,1302fd16bdfe85e523940c846370ef2c,construcao_ferramentas_iluminacao,297.00,delivered,ATRASADO,SP,363.31,boleto,66.31
115899,2017-08-01,7b905861d7c825891d6347454ea7863f,fb55982be901439613a95940feefd9ee,papelaria,79.00,delivered,ADIANTADO,MG,96.80,credit_card,17.80
115900,2017-11-01,32609bbb3dd69b3c066a6860554a77bf,848ef0a103ee21407d1fbd0f7e7ae179,automotivo,39.99,invoiced,NAO IDENTIFICADO,SP,47.77,credit_card,7.78
115901,2018-08-01,b8b61059626efa996a60be9bb9320e10,e9c1abcc7126360336d8f56d49041a8c,beleza_saude,349.00,delivered,ATRASADO,RJ,369.54,credit_card,20.54


In [70]:
df_olist["status_entrega"].value_counts(normalize=True)

status_entrega
ATRASADO              0.712734
ADIANTADO             0.178080
DENTRO DO ESPERADO    0.087642
NAO IDENTIFICADO      0.021544
Name: proportion, dtype: float64

In [71]:
df_olist['order_status'].value_counts(normalize=True)

order_status
delivered      0.978465
shipped        0.010474
canceled       0.004754
processing     0.003123
invoiced       0.003097
unavailable    0.000060
approved       0.000026
Name: proportion, dtype: float64

In [72]:
df_olist.groupby('product_category_name').agg({'product_id': 'nunique'})

,product_id
product_category_name,
agro_industria_e_comercio,74
alimentos,82
alimentos_bebidas,104
artes,55
artes_e_artesanato,19
...,...
sinalizacao_e_seguranca,93
tablets_impressao_imagem,9
telefonia,1134


In [75]:
# Qual o meio de pagamento mais utilizado?

df_olist.groupby('payment_type').agg({'order_id': 'nunique'})

,order_id
payment_type,
boleto,19351
credit_card,74915
debit_card,1494
voucher,3708
